This notebook is part of the tutorial on large scale IE under privacy constraints.

Read files from directory. This is expected to be an html file representing email.

In [4]:
def read_file(file_path): 
    with open(file_path, 'r') as f: 
        return (f.read()) 

In [5]:
file_html = []

In [6]:
import os
# Folder Path
Dir_name = r"D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles"
os.chdir(Dir_name) 
# iterate through all file 
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith(".eml"): 
        file_name = f"{Dir_name}\{file}"
        print(file_name)
        file_html.append(read_file(file_name)) 

D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\Booking confirmed for Jaipur To Hyderabad, Booking Id_ FF20121711252162.eml
D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\Confirmation Email (1).eml
D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\Confirmation Email (2).eml
D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\Confirmation Email (3).eml
D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\Confirmation Email (4).eml
D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\Hyderabad to Indore Flight E-Ticket.eml
D:\Technical\emailExtraction\tutorial\ecir21\excercise\emailFiles\MakeMyTrip Customer ETicket _ NF72695162380045.eml


In [7]:
len(file_html)

7

In [8]:
file_text = []

Clean the email from HTML format.

In [9]:
import sys
from bs4 import BeautifulSoup

from html.parser import HTMLParser
def clean_html():
    for fileTextIns in file_html:
        soup = BeautifulSoup(fileTextIns, "html.parser")
        if soup.body:
            for child in soup.body.children:
                if child.name == 'script':
                    child.decompose() 
            file_text.append(soup.body.get_text('\n'))

Extract all dates from a given text. For different

In [10]:
def extract_date(text):
    months ="January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec"
    years = "([\d]{4}|'[\d]{2})"
    pattern1 = r"(([\d]{1,2})/([\d]{1,2})/("+years+"))"
    pattern2 = r"(([\d]{1,2}\s("+ months +")[,.-]?\s"+years+"))"
    pattern3 = r"((("+ months +")\s[\d]{1,2}[,.-]?\s"+years+"))"
    pattern4 = r"(((0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/(\d\d))|[012][0-9][0-9][0-9]-[01][0-9]-[0123]?[0-9](t|T)[012][0-9]:[0-5][0-9](:[0-5][0-9])?(.[0-9]{3})?([+-][012][0-9](:)?[0-5][0-9])?Z?)"
    return re.finditer(pattern1 + "|" + pattern2 + "|" + pattern3 + "|" + pattern4, text, re.IGNORECASE)

In [11]:
import re
import datetime
from datetime import date
#date_reg_exp = r'((\d{2}-([A-Z]{3}|\d{2})-(\d{4}|\d{2}))|([a-zA-Z]{3}\s\d{1,2},\s\d{4}))'
#all regexs for datetime
#matchDateObj = r'(([\d]{1,2})/([\d]{1,2})/([\d]{4}))|(([\d]{1,2}\s(January|February|March|April|May|June|July|August|September|October|November|December)\s[\d]{4}))|(((0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/(\d\d))|[012][0-9][0-9][0-9]-[01][0-9]-[0123]?[0-9](t|T)[012][0-9]:[0-5][0-9](:[0-5][0-9])?(.[0-9]{3})?([+-][012][0-9](:)?[0-5][0-9])?Z?)'
#date_reg_exp2 = re.compile(matchDateObj)
#date_reg_exp2 = re.compile(r'(\d{2}(/|-|\.)\w{3}(/|-|\.)\d{4})|([a-zA-Z]{3}\s\d{2}(,|-|\.|,)?\s\d{4})|(\d{2}(/|-|\.)\d{2}(/|-|\.)\d+)')
i=0;
clean_html()
for fileTextIns in file_text:
    print(i)
    i=i+1
    matchDateObj = extract_date(fileTextIns)
    #matchDateObj = re.search(date_reg_exp2, fileTextIns)
    
    if matchDateObj:
        for Date in matchDateObj:
            print(Date)

0
<re.Match object; span=(536, 548), match='Dec 17, 2020'>
1
<re.Match object; span=(803, 814), match='2 Dec, 2019'>
<re.Match object; span=(843, 854), match='2 Dec, 2019'>
2
<re.Match object; span=(801, 813), match='18 Aug, 2019'>
<re.Match object; span=(839, 851), match='18 Aug, 2019'>
<re.Match object; span=(1693, 1705), match='18 Aug, 2019'>
<re.Match object; span=(1734, 1746), match='18 Aug, 2019'>
3
<re.Match object; span=(893, 905), match='12 Jan, 2019'>
<re.Match object; span=(934, 946), match='12 Jan, 2019'>
4
<re.Match object; span=(893, 905), match='12 Jan, 2019'>
<re.Match object; span=(934, 946), match='12 Jan, 2019'>
5
<re.Match object; span=(817, 828), match='27 Dec 2020'>
<re.Match object; span=(1060, 1071), match='27 Dec 2020'>
6


Features:
1. Is it a min Distance between checkin keyword and the date. Ex. checkin|check-in|departure|depart|departs|leave|leaves|check\sin|travel
2. Is it a Min Distance between checkout keyword and the date. Ex. checkout|check-out|arrival|arrives|reaches|leaves|check\sout
3. Format of the date and the closest checkin keyword-- are they same or not
4. Format of the date and the closest checkout keyword-- are they same or not
5. Is it a Last but min Distance between checkin keyword and the date. Ex. checkin|check-in|departure|depart|departs|leave|leaves|check\sin|travel
6. Is it a Last but min Distance between checkout keyword and the date. Ex. checkout|check-out|arrival|arrives|reaches|leaves|check\sout
7. Are dates and keywords part of same \<text\> tag
8. Min distance to airline code
9. Min distance to any time
10. Distance to the city names

    

In [12]:
def extract_checkin(text):
    matchCheckinObj = re.search( r'checkin|check-in|departure|depart|departs|leave|leaves|check\sin|travel\s+date', text,re.IGNORECASE)
    return matchCheckinObj

In [30]:
def extract_checkout(text):
    matchCheckoutObj = re.search( r'checkout|check-in|arrival|arrive|arrives|check\sout', text,re.IGNORECASE)
    return matchCheckoutObj

In [13]:
import math
def get_distance(x1, y1, x2, y2):
 return math.sqrt((x1-x2)* (x1-x2) + (y1-y2)*(y1-y2))

In [14]:
i=0;
for fileTextIns in file_text:
    print(i)
    i=i+1
    mindist = sys.maxsize
    matchCheckinObj = extract_checkin(fileTextIns)
    matchDateObj = extract_date(fileTextIns)
    obj = matchCheckinObj
    #matchDateObj = re.search(date_reg_exp2, fileTextIns)
    if matchCheckinObj:
        if matchDateObj:
            for Date in matchDateObj:
                d = get_distance(matchCheckinObj.start(), matchCheckinObj.end(), Date.start(), Date.end())
                if(d < mindist):
                    mindist = d
                    obj = Date
            print(Date)

0
1
<re.Match object; span=(843, 854), match='2 Dec, 2019'>
2
<re.Match object; span=(1734, 1746), match='18 Aug, 2019'>
3
<re.Match object; span=(934, 946), match='12 Jan, 2019'>
4
<re.Match object; span=(934, 946), match='12 Jan, 2019'>
5
<re.Match object; span=(1060, 1071), match='27 Dec 2020'>
6


In [15]:
import numpy as np

In [39]:
def rank_of_dates_with_checkin(fileText):
    matchCheckinObj = extract_checkin(fileTextIns)
    matchDateObj = extract_date(fileTextIns)
    distances = []
    if matchCheckinObj:
        if matchDateObj:
            for Date in matchDateObj:
                d = get_distance(matchCheckinObj.start(), matchCheckinObj.end(), Date.start(), Date.end())
                distances.append(d)
    np_lst = np.array(distances)
    #print(np_lst)
    return np.argsort(np_lst)

In [44]:
import datetime
from datetime import date
#Need to convert extracted dates into datetime objects and compare them
def rank_of_dates(fileText):
    matchDateObj = extract_date(fileTextIns)
    if matchDateObj:
        dates=[]
        for Date in matchDateObj:
            date = datetime.datetime.strptime(Date.group(), '%Y-%m-%d').date()
            dates.append(Date)
    np_lst = np.array(dates)
    #print(np_lst)
    return np.argsort(np_lst)

In [45]:
def rank_of_dates_with_checkout(fileText):
    matchCheckoutObj = extract_checkout(fileTextIns)
    matchDateObj = extract_date(fileTextIns)
    distances = []
    if matchCheckoutObj:
        if matchDateObj:
            for Date in matchDateObj:
                d = get_distance(matchCheckoutObj.start(), matchCheckoutObj.end(), Date.start(), Date.end())
                distances.append(d)
    np_lst = np.array(distances)
    print(np_lst)
    return np.argsort(np_lst)

In [46]:
i=0;
for fileTextIns in file_text:
    print(i)
    i=i+1
    print(rank_of_dates_with_checkin(fileTextIns))

0
[]
1
[1 0]
2
[3 2 1 0]
3
[1 0]
4
[1 0]
5
[0 1]
6
[]


In [47]:
i=0;
for fileTextIns in file_text:
    print(i)
    i=i+1
    print(rank_of_dates(fileTextIns))

0


ValueError: time data 'Dec 17, 2020' does not match format '%Y-%m-%d'

In [38]:
i=0;
for fileTextIns in file_text:
    print(i)
    i=i+1
    print(rank_of_dates_with_checkout(fileTextIns))

0
[]
[]
1
[3567.35434181 3510.78580947]
[1 0]
2
[4671.14825284 4617.40814744 3409.67007202 3351.68733625]
[3 2 1 0]
3
[3039.1462617  2981.16353124]
[1 0]
4
[3039.1462617  2981.16353124]
[1 0]
5
[111.03603019 454.67460892]
[0 1]
6
[]
[]


In [ ]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split